# Database AI Agents: Text-to-SQL for Finance Operations

**Objective**: Build an AI agent that converts natural language questions into safe SQL queries for finance operations.

**Key Features**:
- 🔍 Natural language to SQL conversion
- 🛡️ Safety guardrails (read-only, whitelisted tables, limits)
- ⏰ Time window enforcement for transaction queries
- 🔗 SQLAlchemy integration with PostgreSQL/SQLite
- 📊 Professional financial summaries

**Time**: ~15-20 minutes

**Scenario**: Support a Finance Ops team with quick answers about corporate card spend without writing SQL.

In [20]:
# Import required libraries
import os
import re
import json
from datetime import datetime, timedelta
from dataclasses import dataclass
from typing import List, Optional, Dict, Any, Tuple
import pandas as pd
from sqlalchemy import create_engine, text, MetaData, Table
from sqlalchemy.exc import SQLAlchemyError
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI(
    base_url="https://openai.vocareum.com/v1",
    api_key=os.getenv("OPENAI_API_KEY")
)

print("🔧 Environment Setup:")
print(f"   ✅ OpenAI API Key: {'✓ Configured' if os.getenv('OPENAI_API_KEY') else '❌ Missing'}")
print(f"   🔧 Database: {'✓ Will use SQLite for demo' if not os.getenv('DATABASE_URL') else '✓ PostgreSQL configured'}")

🔧 Environment Setup:
   ✅ OpenAI API Key: ✓ Configured
   🔧 Database: ✓ Will use SQLite for demo


In [21]:
# Data models for our text-to-SQL system

@dataclass
class QueryResult:
    """Represents the result of a text-to-SQL operation"""
    original_question: str
    generated_sql: str
    executed_sql: str
    data: pd.DataFrame
    row_count: int
    summary: str
    time_filter_applied: Optional[str] = None
    assumptions_made: Optional[List[str]] = None
    
    def __post_init__(self):
        if self.assumptions_made is None:
            self.assumptions_made = []

@dataclass
class DatabaseSchema:
    """Represents our known database schema for validation"""
    tables: Dict[str, List[str]]
    relationships: Dict[str, str]
    time_columns: Dict[str, str]

# Define our finance database schema
FINANCE_SCHEMA = DatabaseSchema(
    tables={
        'employees': ['employee_id', 'full_name', 'department', 'cost_center'],
        'cards': ['card_id', 'employee_id', 'last4', 'status'],
        'merchants': ['merchant_id', 'merchant_name', 'category'],
        'transactions': ['txn_id', 'card_id', 'merchant_id', 'txn_time', 'amount_usd', 'currency_code', 'city', 'channel'],
        'departments': ['department', 'cost_center_manager']
    },
    relationships={
        'cards.employee_id': 'employees.employee_id',
        'transactions.card_id': 'cards.card_id',
        'transactions.merchant_id': 'merchants.merchant_id',
        'employees.department': 'departments.department'
    },
    time_columns={
        'transactions': 'txn_time'
    }
)

print("📋 Database Schema Loaded:")
for table, columns in FINANCE_SCHEMA.tables.items():
    print(f"   📊 {table}: {len(columns)} columns")

📋 Database Schema Loaded:
   📊 employees: 4 columns
   📊 cards: 4 columns
   📊 merchants: 3 columns
   📊 transactions: 8 columns
   📊 departments: 2 columns


In [22]:
# Database connection and schema utilities

def get_schema_description(schema: DatabaseSchema) -> str:
    """Get a formatted description of database schema for LLM"""
    desc = "Available Tables and Columns:\n"
    
    for table, columns in schema.tables.items():
        desc += f"\n{table}:\n"
        for col in columns:
            desc += f"  - {col}\n"
    
    desc += "\nKey Relationships:\n"
    for rel, target in schema.relationships.items():
        desc += f"  - {rel} → {target}\n"
        
    desc += "\nTime Columns (for filtering):\n"
    for table, time_col in schema.time_columns.items():
        desc += f"  - {table}.{time_col}\n"
            
    return desc

def check_database_exists():
    """Check if the finance database exists and show stats"""
    db_path = "finance_demo.db"
    if not os.path.exists(db_path):
        print(f"❌ Database file '{db_path}' not found!")
        print("   Run 'python setup_database.py' first to create the database.")
        return False
    
    # Connect and show stats
    engine = create_engine(f"sqlite:///{db_path}", echo=False)
    with engine.connect() as conn:
        # Get table counts
        tables_info = []
        for table in FINANCE_SCHEMA.tables.keys():
            result = conn.execute(text(f"SELECT COUNT(*) FROM {table}"))
            count = result.fetchone()[0]
            tables_info.append(f"   📊 {table}: {count:,} records")
        
        # Get total transaction volume
        result = conn.execute(text("SELECT SUM(amount_usd), COUNT(*) FROM transactions"))
        total_amount, txn_count = result.fetchone()
        
        print("✅ Database connection successful!")
        print("📋 Database Statistics:")
        for info in tables_info:
            print(info)
        print(f"   💰 Total transaction volume: ${total_amount:,.2f}")
        print(f"   📅 Data range: Last 120 days of realistic financial data")
    
    return engine

# Initialize database connection
print("🔗 Connecting to finance database...")
engine = check_database_exists()

if engine:
    print("🤖 Ready to initialize the Finance Text-to-SQL Agent!")
else:
    print("⚠️  Please run the database setup script first.")

🔗 Connecting to finance database...
✅ Database connection successful!
📋 Database Statistics:
   📊 employees: 15 records
   📊 cards: 15 records
   📊 merchants: 20 records
   📊 transactions: 695 records
   📊 departments: 8 records
   💰 Total transaction volume: $572,677.97
   📅 Data range: Last 120 days of realistic financial data
🤖 Ready to initialize the Finance Text-to-SQL Agent!


In [23]:
class FinanceTextToSQLAgent:
    """AI agent for converting natural language to safe SQL queries for finance operations"""
    
    def __init__(self, engine, schema: DatabaseSchema):
        self.engine = engine
        self.schema = schema
        self.query_history = []
        
    def process_question(self, question: str, show_sql_answer: bool = False) -> QueryResult:
        """
        Main method to process a natural language question
        
        Args:
            question: Natural language question about finance data
            show_sql_answer: Whether to display SQL queries during processing
            
        Returns:
            QueryResult with SQL, data, and summary
        """
        print(f"🔍 Processing: {question}")
        
        # Step 1: Generate SQL from natural language with retry logic
        generated_sql, generation_attempts = self._generate_sql_with_retry(question, show_sql_answer)
        if show_sql_answer:
            print(f"📝 Generated SQL (attempt {generation_attempts}): {generated_sql}")
        
        # Step 2: Apply safety checks and modifications
        safe_sql, assumptions = self._apply_safety_checks(generated_sql, question)
        if show_sql_answer:
            print(f"🛡️ Safe SQL: {safe_sql}")
        
        # Step 3: Execute the query
        data, row_count = self._execute_query(safe_sql)
        
        # Step 4: Generate summary
        summary = self._generate_summary(question, safe_sql, data, assumptions)
        
        # Create and store result
        result = QueryResult(
            original_question=question,
            generated_sql=generated_sql,
            executed_sql=safe_sql,
            data=data,
            row_count=row_count,
            summary=summary,
            assumptions_made=assumptions
        )
        
        self.query_history.append(result)
        return result
    
    def _generate_sql_with_retry(self, question: str, show_sql_answer: bool = False, max_attempts: int = 3) -> Tuple[str, int]:
        """
        Generate SQL with retry logic and error feedback
        
        Args:
            question: Natural language question
            show_sql_answer: Whether to show SQL generation details
            max_attempts: Maximum retry attempts
            
        Returns:
            Tuple of (final_sql, attempts_used)
        """
        last_error = None
        
        for attempt in range(1, max_attempts + 1):
            try:
                # Generate SQL with optional error feedback
                sql = self._generate_sql(question, previous_error=last_error, attempt=attempt)
                
                # Test the generated SQL with a quick validation
                validation_error = self._validate_sql_syntax(sql)
                
                if validation_error is None:
                    if attempt > 1 and show_sql_answer:
                        print(f"✅ SQL generation successful on attempt {attempt}")
                    return sql, attempt
                else:
                    last_error = validation_error
                    if show_sql_answer:
                        print(f"❌ Attempt {attempt} failed: {validation_error}")
                        if attempt < max_attempts:
                            print(f"🔄 Retry attempt {attempt + 1} with error feedback...")
                    
            except Exception as e:
                last_error = f"Generation error: {str(e)}"
                if show_sql_answer:
                    print(f"❌ Attempt {attempt} failed: {last_error}")
        
        # If all attempts failed, return the last generated SQL anyway
        if show_sql_answer:
            print(f"⚠️  All {max_attempts} attempts failed, using last attempt")
        return sql, max_attempts
    
    
    def _generate_sql(self, question: str, previous_error: Optional[str] = None, attempt: int = 1) -> str:
        """Generate SQL query from natural language using LLM with optional error feedback"""
        
        schema_info = get_schema_description(self.schema)
        
        # Base prompt for all attempts
        base_rules = """Important Rules:
1. Only use SELECT statements (no INSERT, UPDATE, DELETE, DROP, etc.)
2. Only query from the tables listed above
3. Always include a LIMIT clause (max 20 rows)
4. For transaction queries, always include a time filter on txn_time
5. Use proper JOINs to get related data
6. Use meaningful column aliases for readability
7. Order results logically (e.g., by amount DESC for spending queries)
8. **CRITICAL: Use SQLite functions ONLY - NO MySQL/PostgreSQL syntax**

SQLite Date/Time Functions (USE THESE):
- Time filters: txn_time >= datetime('now', '-30 days')
- Extract month: strftime('%m', txn_time) AS month
- Extract year: strftime('%Y', txn_time) AS year
- Extract date: date(txn_time) AS transaction_date
- Month name: strftime('%B', txn_time) AS month_name

FORBIDDEN Functions (DO NOT USE):
- MONTH() ❌ Use strftime('%m', txn_time) ✅
- YEAR() ❌ Use strftime('%Y', txn_time) ✅  
- NOW() ❌ Use datetime('now') ✅
- INTERVAL ❌ Use datetime('now', '-X days') ✅
- DATE_FORMAT() ❌ Use strftime() ✅"""
        
        # Add error feedback for retry attempts
        error_feedback = ""
        if previous_error and attempt > 1:
            error_feedback = f"\nPREVIOUS ATTEMPT FAILED with error: {previous_error}\nFix the previous error and generate a corrected SQL query.\n"
        
        prompt = f"""You are a SQL expert helping finance operations teams. Convert this natural language question into a SELECT SQL query.

Database Schema:
{schema_info}

{error_feedback}{base_rules}

Question: {question}

Return only the SQL query, no explanations or markdown formatting:"""
        
        try:
            response = client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": prompt},
                    {"role": "user", "content": question}
                ],
                temperature=0.1,
                max_tokens=500
            )
            
            sql = response.choices[0].message.content.strip()
            
            # Clean up formatting
            sql = re.sub(r'```sql\n?', '', sql)
            sql = re.sub(r'```\n?', '', sql)
            
            return sql
            
        except Exception as e:
            error_msg = f"Error generating SQL (attempt {attempt})" if attempt > 1 else "Error generating SQL"
            print(f"❌ {error_msg}: {e}")
            return "SELECT 'Error generating SQL' as error_message LIMIT 1;"
    
    def _validate_sql_syntax(self, sql: str) -> Optional[str]:
        """
        Quick validation of SQL syntax and common issues
        
        Returns:
            None if valid, error message if invalid
        """
        sql_upper = sql.upper().strip()
        
        # Check for common MySQL/PostgreSQL syntax issues
        if 'MONTH(' in sql_upper or 'YEAR(' in sql_upper:
            return "Invalid function: Use strftime() instead of MONTH()/YEAR()"
        
        if 'NOW()' in sql_upper and 'INTERVAL' in sql_upper:
            return "Invalid syntax: Use datetime('now', '-X days') instead of NOW() - INTERVAL"
        
        if 'DATE_FORMAT(' in sql_upper:
            return "Invalid function: Use strftime() instead of DATE_FORMAT()"
        
        # Check for basic structure
        if not sql_upper.startswith('SELECT'):
            return "Query must start with SELECT"
        
        # Check for required elements in transaction queries
        if 'TRANSACTIONS' in sql_upper and 'LIMIT' not in sql_upper:
            return "Missing LIMIT clause for transaction query"
        
        return None
    
    def _apply_safety_checks(self, sql: str, question: str) -> Tuple[str, List[str]]:
        """Apply safety checks and modifications to the generated SQL"""
        
        assumptions = []
        sql_upper = sql.upper().strip()
        
        # 1. Ensure it's a SELECT statement
        if not sql_upper.startswith('SELECT'):
            return "SELECT 'Error: Only SELECT queries are allowed' as error_message;", ["Query rejected - only SELECT allowed"]
        
        # 2. Check for forbidden keywords
        forbidden = ['INSERT', 'UPDATE', 'DELETE', 'DROP', 'CREATE', 'ALTER', 'EXEC']
        for keyword in forbidden:
            if keyword in sql_upper:
                return f"SELECT 'Error: {keyword} operations not allowed' as error_message;", [f"Query rejected - {keyword} not allowed"]
        
        # 3. Ensure LIMIT is present
        if 'LIMIT' not in sql_upper:
            sql = sql.rstrip(';') + ' LIMIT 20;'
            assumptions.append("Added LIMIT 20 for performance")
        
        # 4. Add time filter for transaction queries if missing
        if 'transactions' in sql.lower() and 'txn_time' not in sql.lower():
            where_clause = "txn_time >= datetime('now', '-90 days')"
            
            if 'WHERE' in sql_upper:
                sql = sql.replace('WHERE', f'WHERE {where_clause} AND ', 1)
            else:
                # Insert WHERE clause before ORDER BY or LIMIT
                if 'ORDER BY' in sql_upper:
                    sql = sql.replace('ORDER BY', f'WHERE {where_clause} ORDER BY', 1)
                elif 'LIMIT' in sql_upper:
                    sql = sql.replace('LIMIT', f'WHERE {where_clause} LIMIT', 1)
                else:
                    sql = sql.rstrip(';') + f' WHERE {where_clause};'
            
            assumptions.append("Applied default 90-day time filter for transactions")
        
        return sql, assumptions
    
    def _execute_query(self, sql: str) -> Tuple[pd.DataFrame, int]:
        """Execute SQL query and return results as DataFrame"""
        
        try:
            with self.engine.connect() as conn:
                result = conn.execute(text(sql))
                df = pd.DataFrame(result.fetchall(), columns=result.keys())
                row_count = len(df)
                
                print(f"📊 Query executed: {row_count} rows returned")
                return df, row_count
                
        except SQLAlchemyError as e:
            print(f"❌ Database error: {e}")
            error_df = pd.DataFrame({'error': [f"Database error: {str(e)}"]})
            return error_df, 0
        except Exception as e:
            print(f"❌ Execution error: {e}")
            error_df = pd.DataFrame({'error': [f"Execution error: {str(e)}"]})
            return error_df, 0
    
    def _generate_summary(self, question: str, sql: str, data: pd.DataFrame, assumptions: List[str]) -> str:
        """Generate natural language summary of query results"""
        
        if 'error' in data.columns:
            return f"Query failed: {data['error'].iloc[0]}"
        
        row_count = len(data)
        summary_stats = self._get_data_summary(data)
        
        prompt = f"""You are a financial analyst summarizing query results for a finance operations team.

Original Question: {question}
SQL Executed: {sql}
Rows Returned: {row_count}
Data Summary: {summary_stats}
Assumptions: {', '.join(assumptions) if assumptions else 'None'}

Write a 2-4 sentence professional summary that:
1. Describes what was analyzed
2. Mentions time filters or assumptions made
3. Highlights key insights from the results
4. Uses clear language for finance operations staff

Summary:"""
        
        try:
            response = client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": prompt},
                    {"role": "user", "content": "Generate the summary."}
                ],
                temperature=0.3,
                max_tokens=200
            )
            
            return response.choices[0].message.content.strip()
            
        except Exception as e:
            print(f"❌ Error generating summary: {e}")
            assumptions_text = f" (Assumptions: {', '.join(assumptions)})" if assumptions else ""
            return f"Query returned {row_count} rows{assumptions_text}. Review results for insights."
    
    def _get_data_summary(self, data: pd.DataFrame) -> str:
        """Get summary statistics for LLM context"""
        
        if data.empty:
            return "No data returned"
        
        stats = []
        
        # Amount columns
        amount_cols = [col for col in data.columns if 'amount' in col.lower() or 'spend' in col.lower()]
        for col in amount_cols:
            if data[col].dtype in ['float64', 'int64']:
                total = data[col].sum()
                avg = data[col].mean()
                stats.append(f"{col} total: ${total:,.2f}, average: ${avg:,.2f}")
        
        # Categorical columns
        categorical_cols = [col for col in data.columns if data[col].dtype == 'object']
        for col in categorical_cols[:2]:
            unique_count = data[col].nunique()
            stats.append(f"{col}: {unique_count} unique values")
        
        return "; ".join(stats) if stats else "Mixed data types"

# Initialize the agent
agent = FinanceTextToSQLAgent(engine, FINANCE_SCHEMA)
print("🤖 Finance Text-to-SQL Agent initialized and ready!")

🤖 Finance Text-to-SQL Agent initialized and ready!


In [24]:
# Utility function for displaying results

def display_result(result: QueryResult, show_sql: bool = True):
    """Display query result in a formatted, professional way"""
    
    print("=" * 80)
    print("📊 FINANCE DATABASE QUERY RESULT")
    print("=" * 80)
    
    print(f"\n🔍 Question:")
    print(f"   {result.original_question}")
    
    if show_sql:
        print(f"\n📝 Executed SQL:")
        print(f"   {result.executed_sql}")
    
    if result.assumptions_made:
        print(f"\n⚠️ Assumptions Made:")
        for assumption in result.assumptions_made:
            print(f"   • {assumption}")
    
    print(f"\n📊 Results ({result.row_count} rows):")
    if not result.data.empty and 'error' not in result.data.columns:
        # Format the display nicely
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', None)
        pd.set_option('display.max_colwidth', 30)
        print(result.data.to_string(index=False, max_rows=20))
    else:
        print("   No data returned or error occurred")
    
    print(f"\n💡 Summary:")
    print(f"   {result.summary}")
    
    print("=" * 80)

print("✅ Display utilities loaded")

✅ Display utilities loaded


## 🧪 Test Cases: Natural Language to SQL

Let's test our agent with comprehensive finance questions using our realistic dataset. The database contains 695+ transactions across 15 employees, 20 merchants, and 8 departments over the past 120 days.

In [25]:
# Test Case 1: Top merchants by spend
print("🧪 Test Case 1: Top merchants by total spend")
result1 = agent.process_question("Show me the top 10 merchants by total spend in the last 120 days", show_sql_answer=True)
display_result(result1, show_sql=True)  # Show SQL to debug

🧪 Test Case 1: Top merchants by total spend
🔍 Processing: Show me the top 10 merchants by total spend in the last 120 days
📝 Generated SQL (attempt 1): SELECT 
    merchants.merchant_name, 
    SUM(transactions.amount_usd) AS total_spend 
FROM 
    transactions 
JOIN 
    merchants ON transactions.merchant_id = merchants.merchant_id 
WHERE 
    transactions.txn_time >= datetime('now', '-120 days') 
GROUP BY 
    merchants.merchant_name 
ORDER BY 
    total_spend DESC 
LIMIT 
    10;
🛡️ Safe SQL: SELECT 
    merchants.merchant_name, 
    SUM(transactions.amount_usd) AS total_spend 
FROM 
    transactions 
JOIN 
    merchants ON transactions.merchant_id = merchants.merchant_id 
WHERE 
    transactions.txn_time >= datetime('now', '-120 days') 
GROUP BY 
    merchants.merchant_name 
ORDER BY 
    total_spend DESC 
LIMIT 
    10;
📊 Query executed: 10 rows returned
📝 Generated SQL (attempt 1): SELECT 
    merchants.merchant_name, 
    SUM(transactions.amount_usd) AS total_spend 
FROM 
    tr

In [31]:
# Test Case 2: Department spending analysis
print("\n🧪 Test Case 2: Number of employees by department")
result2 = agent.process_question("Show the number of employees by department", show_sql_answer=True)
display_result(result2, show_sql=False)  # Hide SQL for cleaner output


🧪 Test Case 2: Number of employees by department
🔍 Processing: Show the number of employees by department
📝 Generated SQL (attempt 1): SELECT department, COUNT(employee_id) AS number_of_employees
FROM employees
GROUP BY department
LIMIT 20;
🛡️ Safe SQL: SELECT department, COUNT(employee_id) AS number_of_employees
FROM employees
GROUP BY department
LIMIT 20;
📊 Query executed: 8 rows returned
📝 Generated SQL (attempt 1): SELECT department, COUNT(employee_id) AS number_of_employees
FROM employees
GROUP BY department
LIMIT 20;
🛡️ Safe SQL: SELECT department, COUNT(employee_id) AS number_of_employees
FROM employees
GROUP BY department
LIMIT 20;
📊 Query executed: 8 rows returned
📊 FINANCE DATABASE QUERY RESULT

🔍 Question:
   Show the number of employees by department

📊 Results (8 rows):
 department  number_of_employees
Engineering                    3
    Finance                    2
         HR                    1
      Legal                    1
  Marketing                    2
 Operat

In [27]:
# Test Case 3: High-value transactions
print("\n🧪 Test Case 3: High-value transactions with employee details")
result3 = agent.process_question("All transactions over $1000 in the last 90 days, show employee name, merchant, amount, and card last 4 digits")
display_result(result3)


🧪 Test Case 3: High-value transactions with employee details
🔍 Processing: All transactions over $1000 in the last 90 days, show employee name, merchant, amount, and card last 4 digits
📊 Query executed: 20 rows returned
📊 Query executed: 20 rows returned
📊 FINANCE DATABASE QUERY RESULT

🔍 Question:
   All transactions over $1000 in the last 90 days, show employee name, merchant, amount, and card last 4 digits

📝 Executed SQL:
   SELECT 
    e.full_name AS employee_name, 
    m.merchant_name, 
    t.amount_usd, 
    c.last4 AS card_last_4_digits
FROM 
    transactions t
JOIN 
    cards c ON t.card_id = c.card_id
JOIN 
    employees e ON c.employee_id = e.employee_id
JOIN 
    merchants m ON t.merchant_id = m.merchant_id
WHERE 
    t.txn_time >= datetime('now', '-90 days') 
    AND t.amount_usd > 1000
ORDER BY 
    t.amount_usd DESC
LIMIT 20;

📊 Results (20 rows):
employee_name      merchant_name  amount_usd card_last_4_digits
Ivy Rodriguez             WeWork     4554.19               5

In [28]:
# Test Case 4: Travel expense analysis
print("\n🧪 Test Case 4: Travel expenses by employee")
result4 = agent.process_question("Total travel expenses by employee in the last 120 days, include employee name and department")
display_result(result4)

# Test Case 5: Retry logic demonstration
print("\n🧪 Test Case 5: Complex query that might trigger retry logic")
result5 = agent.process_question("Show quarterly spending trends by month and department with year-over-year comparison", show_sql_answer=True)
display_result(result5, show_sql=False)


🧪 Test Case 4: Travel expenses by employee
🔍 Processing: Total travel expenses by employee in the last 120 days, include employee name and department
📊 Query executed: 15 rows returned
📊 Query executed: 15 rows returned
📊 FINANCE DATABASE QUERY RESULT

🔍 Question:
   Total travel expenses by employee in the last 120 days, include employee name and department

📝 Executed SQL:
   SELECT 
    e.full_name AS employee_name, 
    e.department, 
    SUM(t.amount_usd) AS total_travel_expenses
FROM 
    employees e
JOIN 
    cards c ON e.employee_id = c.employee_id
JOIN 
    transactions t ON c.card_id = t.card_id
JOIN 
    merchants m ON t.merchant_id = m.merchant_id
WHERE 
    m.category = 'Travel' AND
    t.txn_time >= datetime('now', '-120 days')
GROUP BY 
    e.employee_id
ORDER BY 
    total_travel_expenses DESC
LIMIT 20;

📊 Results (15 rows):
employee_name  department  total_travel_expenses
 David Wilson Engineering               19595.48
    Bob Smith   Marketing               14678.20

## 🛡️ Safety Validation Tests

Now let's test our safety guardrails to ensure the agent properly handles potentially dangerous queries:

In [29]:
# Safety Test 1: Attempt forbidden operations
print("🛡️ Safety Test 1: Attempt to DELETE data")
safety_result1 = agent.process_question("Delete all transactions from Alice Johnson")
display_result(safety_result1)

🛡️ Safety Test 1: Attempt to DELETE data
🔍 Processing: Delete all transactions from Alice Johnson
📊 Query executed: 0 rows returned
📊 Query executed: 0 rows returned
📊 FINANCE DATABASE QUERY RESULT

🔍 Question:
   Delete all transactions from Alice Johnson

📝 Executed SQL:
   SELECT transactions.txn_id, transactions.card_id, transactions.merchant_id, transactions.txn_time, transactions.amount_usd, transactions.currency_code, transactions.city, transactions.channel
FROM transactions
JOIN cards ON transactions.card_id = cards.card_id
JOIN employees ON cards.employee_id = employees.employee_id
WHERE employees.full_name = 'Alice Johnson' AND transactions.txn_time >= datetime('now', '-30 days')
LIMIT 20;

📊 Results (0 rows):
   No data returned or error occurred

💡 Summary:
   The executed SQL query was designed to retrieve the transaction details of Alice Johnson from the past 30 days. However, no transactions were found for Alice Johnson in this time frame. Therefore, there were no transa

In [30]:
# Safety Test 2: Query without time filter (should add default)
print("\n🛡️ Safety Test 2: Missing time filter - should add 90-day default")
safety_result2 = agent.process_question("Show all transactions by employee Maya Patel")
display_result(safety_result2)


🛡️ Safety Test 2: Missing time filter - should add 90-day default
🔍 Processing: Show all transactions by employee Maya Patel
📊 Query executed: 0 rows returned
📊 Query executed: 0 rows returned
📊 FINANCE DATABASE QUERY RESULT

🔍 Question:
   Show all transactions by employee Maya Patel

📝 Executed SQL:
   SELECT 
    transactions.txn_id, 
    transactions.card_id, 
    transactions.merchant_id, 
    transactions.txn_time, 
    transactions.amount_usd, 
    transactions.currency_code, 
    transactions.city, 
    transactions.channel
FROM 
    transactions
JOIN 
    cards ON transactions.card_id = cards.card_id
JOIN 
    employees ON cards.employee_id = employees.employee_id
WHERE 
    employees.full_name = 'Maya Patel' AND 
    transactions.txn_time >= datetime('now', '-30 days')
LIMIT 20;

📊 Results (0 rows):
   No data returned or error occurred

💡 Summary:
   The analysis was performed to identify all transactions made by the employee Maya Patel in the last 30 days. The SQL query wa

## 🎯 Exercise Summary

This Database AI Agent demonstrates key concepts for building safe text-to-SQL systems for finance operations:

### ✅ **Core Features Implemented**

1. **Natural Language Processing**: Converts plain English questions to SQL using GPT-4
2. **Safety Guardrails**: Enforces read-only operations, table whitelisting, and row limits  
3. **Time Window Enforcement**: Automatically adds time constraints for transaction queries
4. **Professional Summaries**: Generates clear explanations suitable for finance teams
5. **Database Integration**: SQLAlchemy support for PostgreSQL/SQLite connectivity

### 🛡️ **Security & Safety Measures**

- **Query Validation**: Blocks DML operations (INSERT, UPDATE, DELETE, DROP)
- **Table Whitelisting**: Only allows queries against approved schema tables
- **Automatic Limits**: Adds LIMIT 20 to prevent large result sets
- **Time Constraints**: Requires time filters for transaction queries (90-day default)
- **Error Handling**: Graceful failure with informative error messages

### 🏗️ **Architecture Highlights**

- **Modular Design**: Separate classes for schema, results, and agent logic
- **Schema-Aware**: Understands table relationships and data types
- **Query History**: Tracks all processed queries for auditing and analytics
- **Assumption Tracking**: Records when defaults are applied automatically


### 💡 **Key Learning Outcomes**

Students learn to build AI systems that:
- Safely bridge natural language and database operations
- Implement robust guardrails for production environments
- Handle edge cases and provide meaningful error messages
- Generate professional summaries for business stakeholders
- Track and audit AI-generated database interactions

This foundation enables building sophisticated financial analysis tools that democratize data access while maintaining security and compliance standards.